In [ ]:
# This is the cell to Run SD Next
# Uncomment / change the variables below as you wish

# ========================================================================

# Use pytorch nightly (will override TORCH_COMMAND)
# %env PYTORCH_NIGHTLY=1

# Install specific version of torch
# %env TORCH_COMMAND=""

%env COMMANDLINE_ARGS=--listen

# ========================================================================

!/sdnext-start.sh

In [ ]:
# This is the cell to manage the things you need to download
# Change the variables below as you wish

# It is recommended to run this cell first, if you have anything to download
# You can ignore this if you have nothing to download

# ========================================================================

# There can be either one or two element(s) per array
# first element is for the download link
# the second element is for destination 
# - default to /workspace if this element is not defined
# - relative paths are based on /workspace, example: "./SD Next/" will be "/workspace/SD Next/"
DOWNLOAD_LIST = [
    ["https://civitai.com/api/download/models/432714?type=Model&format=SafeTensor&size=pruned&fp=fp16", "./SD Next/models/Stable-diffusion/"],
    # add more as you wish
    # ["", ""],
    # ["", ""],
    # ["", ""],
]

# There can be either two or three element(s) per array
# first element is for the HF repo
# the second element is for path to file in repo
# the third element is for destination 
# - default to /workspace if this element is not defined
# - relative paths are based on /workspace, example: "./SD Next/" will be "/workspace/SD Next/"
# File structures are maintained (i.e if the file path is XX/YY.txt and destination is ./ABC, the file will be saved as /workspace/ABC/XX/YY.txt)

HUGGINGFACE_LIST = [
    ["syaimu/7th_Layer_XL", "7th_anime_XL-A-fp16.safetensors", "./SD Next/models/Stable-diffusion/"],
    ["LarryAIDraw/autismmixSDXL_autismmixConfetti", "autismmixSDXL_autismmixConfetti.safetensors", "./SD Next/models/Stable-diffusion/"],
    # add more as you wish
    # ["", "", ""],
    # ["", "", ""],
    # ["", "", ""],
]

# ========================================================================

%cd /workspace

import os
from urllib.parse import urlsplit
import requests
from huggingface_hub import hf_hub_download
import humanize
import time

for dl in HUGGINGFACE_LIST:
    repo, filePath = dl[:2]
    dest = dl[2] if len(dl) == 3 else "./"
    resultPath = os.path.join(dest, filePath)
    print(f'\n\nDownloading "{os.path.basename(filePath)}" from "{repo}" to {resultPath}...')
    start = time.time()
    hf_hub_download(repo_id=repo, filename=filePath, local_dir=dest)
    end = time.time()
    timeUsed = end - start
    fileSize = os.path.getsize(resultPath)
    avgSpeed = humanize.naturalsize(fileSize/timeUsed, binary=True)
    fileSize = humanize.naturalsize(fileSize, binary=True)
    print(f'Download completed\n- Time used: {timeUsed}s\n- File size: {fileSize}\n- Average speed: {avgSpeed}/s')

for dl in DOWNLOAD_LIST:
    baseLink = dl[0]
    filename = os.path.basename(urlsplit(baseLink).path)
    
    print(f'\n\nFetching "{baseLink}"...')
    try:
        response = requests.get(baseLink, allow_redirects=True, timeout=10, stream=True)
        finalLink = response.url
        code = response.status_code 
        if code >= 400:
            print(f'Error occurred while fetching "{baseLink}"')
            print(f'Code - {code}')
            print(f'Message - {response.text}')
            continue
        if code == 302:
            finalLink = response.headers['Location']
            print("Redirect Detected")
        response.close()
    except requests.Timeout:
        print("Error - Timed out")
        continue
    except requests.exceptions.RequestException as e:
        print("An error occurred:", str(e))
        continue
        
    print(f'The link is the source link' if finalLink == baseLink else f'Resolved source link - "{finalLink}"')

    filename = filename if "." in filename else os.path.basename(urlsplit(finalLink).path)
    dest = dl[1] if len(dl) == 2 else "./"
    resultPath = os.path.join(dest, filename)
    dest = f'"{dl[1] if len(dl) == 2 else "./"}"'
    filename = f'"{filename}"'
    baseLink = f'"{baseLink}"'

    print(f'Downloading {filename} to {resultPath}...')
    start = time.time()
    !aria2c -x 16 -s 8 -d $dest -o $filename $baseLink > /dev/null
    end = time.time()
    timeUsed = end - start
    fileSize = os.path.getsize(resultPath)
    avgSpeed = humanize.naturalsize(fileSize/timeUsed, binary=True)
    fileSize = humanize.naturalsize(fileSize, binary=True)
    print(f'Download completed\n- Time used: {timeUsed}s\n- File size: {fileSize}\n- Average speed: {avgSpeed}/s')

print("\n\nDownload finished")

In [ ]:
# This is the cell to manage the git version
# Change the variables below as you wish

# ========================================================================

%env BRANCH=
%env COMMIT=

# ========================================================================

!/sdnext-git.sh